# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить два задания

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 5 баллов

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 5 баллов

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам.


__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
# !pip install wandb
# !pip3 install pytorch_lightning torchmetrics  

**✨ Внимание ✨**

В этом домашнем задании предлагается использовать библиотеку `pytorch_lightning`. Доступ к ее [документации](https://lightning.ai/docs/pytorch/stable/) заблокирован с территории РФ. Вы можете:

1. Получить к ней доступ с помощью VPN.

2. Собрать документацию самостоятельно. Для этого склонируйте [github-репозиторий](https://github.com/Lightning-AI/lightning/tree/master), запустите в нем терминал (на windows – git bash) и выполните команды:

```shell
git submodule update --init --recursive
make docs
```
После этого откройте появившийся файл `docs/build/html/index.html`. Для работы команд в вашем окружении должен быть `pip`. Полная инструкция [по ссылке](https://github.com/Lightning-AI/lightning/tree/master/docs).

3. Гуглить `<error message> pytorch lightning` или `<how to do this> pytorch lightning`. Stack overflow на территории РФ все еще доступен 😉

4. Не пользоваться `pytorch_lightning` и написать цикл обучения модели самостоятельно. Например, по аналогии с функцией `fit` из [семинара 4](https://github.com/hse-ds/iad-deep-learning/blob/master/2023/seminars/04.%20Optim%20%26%20Lightning/04_Optim%26Lightning_solution.ipynb).

In [1]:
# Возможно ниже будет не очень работать WandbLogger с первого разу, перезапустите kernel тетрадки
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: naevseev-work. Use `wandb login --relogin` to force relogin


True

In [2]:
import math
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from pytorch_lightning.loggers import WandbLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy
from torchvision.datasets import ImageFolder
from tqdm import tqdm

# You may add any imports you need

In [3]:
def seed_everything(seed):
    # Фискирует максимум сидов.
    # Это понадобится, чтобы сравнение оптимизаторов было корректным
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(13)

## Задание 0

### Что поможет сделать на 10 из 10 (одно задание - 5 баллов)

1. Использовать все возможные методы оптимизации и эксперемнтировать с ними.
2. Подбор learning rate. Пример из прошлого семинара как это делать: [Как найти lr](https://pytorch-lightning.readthedocs.io/en/1.4.5/advanced/lr_finder.html)

```
  trainer = pl.Trainer(accelerator="gpu", max_epochs=2, auto_lr_find=True) 

  trainer.tune(module, train_dataloader, eval_dataloader)

  trainer.fit(module, train_dataloader, eval_dataloader))
```



3. Аугментация данных. [Документация (полезная)](https://pytorch.org/vision/main/transforms.html), а также [библиотека albumentation](https://towardsdatascience.com/getting-started-with-albumentation-winning-deep-learning-image-augmentation-technique-in-pytorch-47aaba0ee3f8)
4. Подбор архитектуры модели. 
5. Можно написать модель руками свою в YourNet, а можно импортировать не предобученную сетку известной архитектуры из модуля torchvision.models. Один из способов как можно сделать: 

  * `torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)`
  * Документация по возможным моделям и как их можно брать: [Документация (полезная)](https://pytorch.org/vision/stable/models.html)
6. Правильно нормализовывать данные при создании, пример [тык, но тут и в целом гайд от и до](https://www.pluralsight.com/guides/image-classification-with-pytorch)
7. Model Checkpointing. Сохраняйте свой прогресс (модели), чтобы когда что-то пойдет не так вы сможете начать с этого места или просто воспроизвести свои результаты модели, которые обучали. 
 * Пример как можно с wandb тут: [Сохраняем лучшие модели в wandb](https://docs.wandb.ai/guides/integrations/lightning)
 * По простому можно так: [Сохраняем модели в pytorch дока](https://pytorch.org/tutorials/beginner/saving_loading_models.html)

### Подготовка данных

In [6]:
transform_img = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.ImageFolder('/Users/nikitaevseev/Desktop/DS/DL/dataset/dataset/train', transform=transform_img)

train_dataset_loader = DataLoader(
  train_dataset, 
  batch_size=len(train_dataset)
)

In [11]:
def mean_std(loader):
  images, lebels = next(iter(loader))
  mean, std = images.mean([0,2,3]), images.std([0,2,3])
  return mean, std
mean, std = mean_std(train_dataset_loader)
print("mean and std: \n", mean, std)

mean and std: 
 tensor([0.4802, 0.4481, 0.3975]) tensor([0.2770, 0.2691, 0.2821])


In [12]:
# YOU CAN DEFINE AUGMENTATIONS HERE
train_transform = transforms.Compose(
    [
        transforms.RandomApply(transforms=[transforms.ColorJitter(brightness=1.0, contrast=0.5, saturation=1, hue=0.1)], p=0.2),
        transforms.RandomApply(transforms=[transforms.AugMix()], p=0.25),
        transforms.RandomApply(transforms=[transforms.Grayscale(num_output_channels=3)], p=0.07),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2770, 0.2691, 0.2821])
    ]
)
#(0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)                    [0.485, 0.456, 0.406] [0.229, 0.224, 0.225]
val_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2770, 0.2691, 0.2821])
    ]
)

#applier = transforms.RandomApply(transforms=[AugMix], p=0.5)

#transforms = transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(), ]), p=0.3)

train_dataset = ImageFolder('/Users/nikitaevseev/Desktop/DS/DL/dataset/dataset/train', transform=train_transform)
val_dataset = ImageFolder('/Users/nikitaevseev/Desktop/DS/DL/dataset/dataset/val', transform=val_transform)
# REPLACE ./dataset/dataset WITH THE FOLDER WHERE YOU DOWNLOADED AND UNZIPPED THE DATASET

train_dataloader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=7, persistent_workers=True, multiprocessing_context='forkserver')
val_dataloader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=7, persistent_workers=True, multiprocessing_context='forkserver')

In [13]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Посмотрим на картиночки

In [ ]:
for batch in val_dataloader:
    images, class_nums = batch
    plt.imshow(images[5].permute(1, 2, 0))
    plt.show()
    plt.imshow(images[15].permute(1, 2, 0))
    plt.show()
    break

## Задание 1. 

5 баллов
Добейтесь accuracy на валидации не менее 0.44. В этом задании запрещено пользоваться предобученными моделями и ресайзом картинок.


Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались)
4. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

### Модель (или просто импортируйте не предобученную)

In [11]:
class YourNet(torch.nn.Module):
    def __init__(self, dropout):
        super().__init__()

        self.relu = torch.nn.ReLU()

        self.batch_norm1 = torch.nn.BatchNorm2d(16)
        self.batch_norm2 = torch.nn.BatchNorm2d(32)
        self.batch_norm3 = torch.nn.BatchNorm2d(64)
        self.batch_norm4 = torch.nn.BatchNorm1d(128)

        self.maxpool = torch.nn.MaxPool2d(2)

        self.conv1 = torch.nn.Conv2d(3, 16, 3)
        self.conv2 = torch.nn.Conv2d(16, 16, 3)
        self.conv3 = torch.nn.Conv2d(16, 32, 3)
        self.conv4 = torch.nn.Conv2d(32, 32, 3)
        self.conv5 = torch.nn.Conv2d(32, 64, 3)
        self.conv6 = torch.nn.Conv2d(64, 64, 3)

        self.flatten = nn.Flatten()
        self.linear1 = torch.nn.Linear(64 * 4 * 4, 128)
        self.output = torch.nn.Linear(128, 200)

        self.dropout = torch.nn.Dropout(p=dropout)

        # --------------
        self.loss_func = torch.nn.CrossEntropyLoss()
        self.targets = torch.Tensor()
        self.preds = torch.Tensor()
        pass

    def _forward(self, x):
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv5(x)
        x = self.batch_norm3(x)
        x = self.relu(x)
        x = self.conv6(x)
        x = self.batch_norm3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.batch_norm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        return self.output(x)

    def forward(self, images, target=None):
        # images ~ (batch size, num channels, height, width)
        # target ~ (batch size)
        # output ~ (batch size, num classes)
        output = self._forward(images)

        # get accuracy score and save it to self.accuracy
        if target is not None:
            loss = self.loss_func(output, target)

            self.targets = torch.cat((self.targets, target.cpu()), 0)
            pred = torch.argmax(output, dim=-1)
            self.preds = torch.cat((self.preds, pred.cpu()), 0)
            self.accuracy = accuracy(self.preds.long(), self.targets.long(), task='multiclass', num_classes=200)

        return loss if target is not None else output

    def get_accuracy(self, reset=False):
        # return accuracy by all values till now
        if reset:
            self.targets = torch.Tensor()
            self.preds = torch.Tensor()
        return self.accuracy

In [22]:
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel

#processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
#model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

# Загрузите предварительно обученный конфигурационный файл модели
model_name = "google/vit-base-patch16-224"
config = ViTModel.from_pretrained(model_name).config

# Установите свои параметры, например, количество классов и размер изображений
num_classes = 200
image_size = 64

# Создайте новую конфигурацию с вашими параметрами
config.vocab_size = num_classes
config.image_size = (image_size, image_size)

# Создайте новую модель на основе этой конфигурации
model = ViTForImageClassification(config)


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import transformers

# Скачайте модель vit-g/14

model = transformers.ViTForImageClassification.from_pretrained("google/vit-g-14", pretrained=False)

### Тренировочный класс lightning

In [9]:
from lion_pytorch import Lion
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [10]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4, wd=10.0):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.wd = wd

    def forward(self, x):
        result = self.model(x)
        return result

    def configure_optimizers(self):
        #optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate*10)
        optimizer = Lion(self.parameters(), lr=self.learning_rate, weight_decay=self.wd)
        return {
        "optimizer": optimizer,
        "lr_scheduler": {
            "scheduler": ReduceLROnPlateau(optimizer, factor=0.1),
            "monitor": "train_loss",
            "frequency": 1
            },
        }

    def training_step(self, train_batch, batch_idx):
        images, target = train_batch
        output = self.model(images)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )  # сохраняет логи в папку, но можно несложно добавить wandb
        return loss

    def validation_step(self, val_batch, batch_idx):
        images, target = val_batch
        #loss = self.model(images, target)
        output = self.model(images)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        self.log("val_loss", loss, prog_bar=True)

In [ ]:
trainer = pl.Trainer()

# Run learning rate finder
lr_finder = trainer.tuner.lr_find(model)

# Results can be found in
lr_finder.results

In [ ]:
trainer = pl.Trainer(accelerator="gpu", max_epochs=2, auto_lr_find=True) 

trainer.tune(module, train_dataloader, val_dataloader)

trainer.fit(module, train_dataloader, val_dataloader)

In [17]:
#resnet = torchvision.models.shufflenet_v2_x1_5(pretrained=False, num_classes=200)
resnet = torchvision.models.resnet18(pretrained=False, num_classes=200)
#num_classes = 200  # Задайте количество классов для вашей задачи
#resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)

/Users/nikitaevseev/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nikitaevseev/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [18]:
wandb_logger = WandbLogger\
    (log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning

model = resnet#YourNet(dropout=0.4)
module = YourModule(model) # YOUR CODE HERE

# инициализируем проект
wandb.init(project="hw_02_exs1", name="resnet18+Lion+entr+512v3")
# сохраняем параметры сетки в wandb + просим следить за градиентами сетки
wandb.watch(model)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


[]

In [19]:
trainer = pl.Trainer(logger=wandb_logger, max_epochs=8)
trainer.fit(module, train_dataloader, val_dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/nikitaevseev/anaconda3/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.3 M
---------------------------------
11.3 M    Trainable params
0         Non-trainable params
11.3 M    Total params
45.116    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 6.63 GB, other allocations: 2.38 GB, max allowed: 9.07 GB). Tried to allocate 128.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

### Валидация результатов задания

In [16]:
def evaluate_task(model, test_dataloader):
    model = model
    model.eval()
    taccuracy = 0.0
    for images, labels in tqdm(test_dataloader):
        images, labels = images, labels
        with torch.no_grad():
            #loss = model(images, labels)
            output = model(images)
            loss = torch.nn.CrossEntropyLoss()(output, labels)
            targets = labels
            pred = torch.argmax(output, dim=-1)
            preds = pred
            acc_batch = accuracy(preds.long(), targets.long(), task='multiclass', num_classes=200)
        taccuracy += acc_batch
    taccuracy = taccuracy / len(test_dataloader)
    return taccuracy

In [17]:
accuracy_score = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * accuracy_score / 0.44, 0, 10):.2f} баллов")
print(accuracy_score)

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:33<00:00,  1.66s/it]

Оценка за это задание составит 0.75 баллов
tensor(0.0329)


In [ ]:
for images, labels in tqdm(val_dataloader):
        print(labels)

Более глубокие модели ResNet50-200 слишком долго обучаются. То же можно сказать про меньший batch_size, например 64, 128, а также иные оптимизаторы (AdamW). 

## Задание 2

5 баллов
Добейтесь accuracy на валидации не менее 0.84. В этом задании делать ресайз и использовать претрейн можно.

Для того чтобы выбить скор (считается ниже) на 2.5/5 балла (то есть половину за задание) достаточно соблюдать пару простых жизненных правил:
1. Аугментация (без нее сложно очень будет)
2. Оптимайзеры можно (и нужно) использовать друг с другом. Однако когда что-то проверяете, то не меняйте несколько параметров сразу - собьете логику экспериментов
3. Не используйте полносвязные модели или самые первые сверточные, используйте более современные архитектуры (что на лекциях встречались или можете пойти дальше).
4. Попробуйте сначала посмотреть качество исходной модели без дообучения, сохраните как baseline. Отсюда поймете какие слои нужно дообучать.
5. Посмотреть все ноутбуки прошедших семинаров и слепить из них что-то общее. Семинарских тетрадок хватит сверх

In [5]:
transform_img = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.ImageFolder('/Users/nikitaevseev/Desktop/DS/DL/dataset/dataset/train', transform=transform_img)

train_dataset_loader = DataLoader(
  train_dataset, 
  batch_size=len(train_dataset)
)

In [5]:
# YOU CAN DEFINE AUGMENTATIONS HERE
train_transform = transforms.Compose(
    [
        transforms.RandomApply(transforms=[transforms.ColorJitter(brightness=1.0, contrast=0.5, saturation=1, hue=0.1)], p=0.2),
        transforms.Resize((224, 224)),
        transforms.RandomApply(transforms=[transforms.AugMix()], p=0.25),
        transforms.RandomApply(transforms=[transforms.Grayscale(num_output_channels=3)], p=0.07),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2770, 0.2691, 0.2821])
    ]
)
#(0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)                    [0.485, 0.456, 0.406] [0.229, 0.224, 0.225]
val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2770, 0.2691, 0.2821])
    ]
)

#applier = transforms.RandomApply(transforms=[AugMix], p=0.5)

#transforms = transforms.RandomApply(torch.nn.ModuleList([transforms.ColorJitter(), ]), p=0.3)

train_dataset = ImageFolder('/Users/nikitaevseev/Desktop/DS/DL/dataset/dataset/train', transform=train_transform)
val_dataset = ImageFolder('/Users/nikitaevseev/Desktop/DS/DL/dataset/dataset/val', transform=val_transform)
# REPLACE ./dataset/dataset WITH THE FOLDER WHERE YOU DOWNLOADED AND UNZIPPED THE DATASET

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=7, persistent_workers=True, multiprocessing_context='forkserver')
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=7, persistent_workers=True, multiprocessing_context='forkserver')

### Модель (или просто импортируйте предобученную)

In [ ]:
class YourNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        pass

    def _forward(self, x):
        # runs the Neural Network
        # YOUR CODE HERE
        pass

    def forward(self, images, target=None):
        # YOUR CODE HERE
        pass

    def get_accuracy(self, reset=False):
        # YOUR CODE HERE
        pass

### Тренировочный класс lightning

In [6]:
class YourModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4, wd=10.0):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.wd = wd

    def forward(self, x):
        result = self.model(x)
        return result

    def configure_optimizers(self):
        #optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate*10)
        optimizer = Lion(self.parameters(), lr=self.learning_rate, weight_decay=self.wd)
        return {
        "optimizer": optimizer,
        "lr_scheduler": {
            "scheduler": ReduceLROnPlateau(optimizer, factor=0.1),
            "monitor": "train_loss",
            "frequency": 1
            },
        }

    def training_step(self, train_batch, batch_idx):
        images, target = train_batch
        output = self.model(images)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        self.log(
            "train_loss", loss, prog_bar=True
        )  # сохраняет логи в папку, но можно несложно добавить wandb
        return loss

    def validation_step(self, val_batch, batch_idx):
        images, target = val_batch
        #loss = self.model(images, target)
        output = self.model(images)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        self.log("val_loss", loss, prog_bar=True)

In [7]:
wandb_logger = WandbLogger\
    (log_model='all') # какие возможности дает с pytorch_lightning https://docs.wandb.ai/guides/integrations/lightning

model = torchvision.models.vit_h_14(pretrained=False, num_classes=200)#YourNet(dropout=0.4)
module = YourModule(model) # YOUR CODE HERE

# инициализируем проект
wandb.init(project="hw_02_exs1", name="vit_h_14+Lion+entr+512")
# сохраняем параметры сетки в wandb + просим следить за градиентами сетки
wandb.watch(model)

/Users/nikitaevseev/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/nikitaevseev/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[]

In [ ]:
trainer = pl.Trainer(logger=wandb_logger, max_epochs=2)
trainer.fit(module, train_dataloader, val_dataloader)

### Валидация результатов задания

In [ ]:
model = # Подгрузить свою сохраненную модель сюда

accuracy = evaluate_task(model, val_dataloader)
print(f"Оценка за это задание составит {np.clip(10 * (accuracy - 0.5) / 0.34, 0, 10):.2f} баллов")

# Отчёт об экспериментах 

текст писать тут (или ссылочку на wandb/любой трекер экспреиментов) для каждого задания, то есть не обязательно именно тут рисовать графики, если вы используете готовые трекеры/мониторинги ваших моделей.